## Scenario 3: building a model trained on both stream gauges and HWMs with considering uncertainties

### Importing necessary libraries

In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings 
import tensorflow as tf
import h5py
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


np.random.seed(1)
tf.random.set_seed(1)

### Reading the stream gauges and HWM data for hurricane Ida

In [2]:
# Read the CSV file into a pandas DataFrame
df_HWM = pd.read_csv('HWMs.csv', header=0)
df_HWM['Mean_Gage_height']= 0
df_HWM['Type'] = 'HWMs'

df_strm = pd.read_csv('Ida.csv', header=0)
df_strm['HAND']=0
df_strm['Distance to river']=0
df_strm['Uncertainty']= 0
df_strm['Flood']= df_strm['Max_Gage_height']-df_strm['Mean_Gage_height']
df_strm['Type'] = 'Stream'

### Combining stream gauge and HWM data

In [3]:
common_columns= [
    'Curvature',
    'Altitude', 
    'HAND', 
    'Storm surge', 
    'Wind speed', 
    'Rain',
    'Soil moisture', 
    'Flow accumulation', 
    'TWI', 
    'Slope',
    'Distance to river',
    'Distance to storm track', 
    'Mean_Gage_height',
    'Distance to sea',
    'Uncertainty',
    'Type',
    'Flood'   
]

# Select common columns from both DataFrames
df_HWM_common = df_HWM[common_columns]
df_strm_common = df_strm[common_columns]


# Concatenate DataFrames
df = pd.concat([df_HWM_common, df_strm_common], ignore_index=True)
df

,Curvature,Altitude,HAND,Storm surge,Wind speed,Rain,Soil moisture,Flow accumulation,TWI,Slope,Distance to river,Distance to storm track,Mean_Gage_height,Distance to sea,Uncertainty,Type,Flood
0,-1.929120e+09,7.37162,8.89971,0.772498,34.5617,0.044960,0.392925,0,-5.933630,2.002040,0.000736,0.391555,0.000000,0.073790,0.1,HWMs,2.050000
1,-5.667340e+09,7.37162,8.74236,0.772498,34.5617,0.044960,0.392925,51,-2.421870,3.105190,0.000719,0.391619,0.000000,0.073833,0.1,HWMs,3.920000
2,-5.007710e+08,3.46794,3.46790,2.017150,33.5172,0.124228,0.331936,25,-0.200457,0.168553,0.015490,0.489294,0.000000,0.062434,0.2,HWMs,1.700000
3,1.010870e+09,3.74653,3.87530,1.589750,33.7572,0.119348,0.331936,0,-3.989910,0.286747,0.000391,0.419288,0.000000,0.133445,0.2,HWMs,2.300000
4,1.526490e+09,5.20485,5.68098,0.772498,34.5617,0.044960,0.392925,68759,6.244250,0.708235,0.000027,0.393270,0.000000,0.074235,0.2,HWMs,1.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,9.700830e+09,315.74000,0.00000,1.282130,31.5753,1.533940,0.279366,2592720,16.286273,12.305600,0.000000,0.985841,3.757292,0.370593,0.0,Stream,2.502708
367,3.441800e+08,270.50000,0.00000,1.296930,31.9977,1.286300,0.352202,2924290,18.546203,1.420260,0.000000,0.974283,2.472708,0.360568,0.0,Stream,5.177292
368,-1.934700e+09,331.84000,0.00000,1.275670,31.7314,1.619680,0.258539,552958,15.547339,5.531940,0.000000,0.916809,2.266146,0.303099,0.0,Stream,2.503854
369,-8.706530e+09,586.60000,0.00000,1.337360,30.7331,0.494721,0.381326,369281,14.192663,14.157900,0.000000,1.100598,1.503437,0.481995,0.0,Stream,2.766563


### Defining features and the target variable

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

features= [
    'Curvature',
    'Altitude', 
    'HAND', 
    'Storm surge', 
    'Wind speed', 
    'Rain',
    'Soil moisture', 
 #   'Flow accumulation', 
 #   'TWI', 
    'Slope',
    'Distance to river',
    'Distance to storm track', 
    'Mean_Gage_height',
    'Uncertainty',
    'Distance to sea'   
]
x = df[features].copy()

df['Uncertainty']=df['Uncertainty']* 0.3048  #ft to m
y = df['Flood']* 0.3048    #ft to m
x.shape

(371, 13)

### Splitting and scaling data

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = scaler.fit_transform(x)


# Split data into training and testing sets
Xtrain, Xtest, Ytrain, Ytest , X_train_indices, X_test_indices= train_test_split(x, y, df.index, test_size=0.2, random_state=64)

### Defining a custom loss function instead of standard one considering uncertainties

In [6]:
# Custom loss function
def custom_loss(y_true, y_pred, uncertainty):
    lower_bound = y_true - uncertainty
    upper_bound = y_true + uncertainty

    # Check if prediction falls within the uncertainty range
    within_range = tf.logical_and(tf.less_equal(y_pred, upper_bound), tf.greater_equal(y_pred, lower_bound))
    
    # Calculate the distance to the nearest edge of the range
    distance_to_range = tf.where(
        within_range,
        tf.zeros_like(y_pred),  # If within range, no penalty
        tf.minimum(tf.abs(y_pred - lower_bound), tf.abs(y_pred - upper_bound))  # Distance to nearest edge
    )
    
    # Define a penalty factor 
    penalty_factor = 1.1
    
    # Calculate the loss as a combination of within range and distance to range
    loss = tf.where(
        within_range,
        tf.zeros_like(y_pred),  # If within range, no penalty
        penalty_factor * distance_to_range
    )
    
    # Return the mean loss
    return tf.reduce_mean(loss)

# Wrapper to handle the uncertainty dynamically, this function should match how you intend to use the model
def loss_wrapper(uncertainty):
    def loss(y_true, y_pred):
        return custom_loss(y_true, y_pred, uncertainty)
    return loss

In [7]:
### Creating a model with custom loss function

In [102]:
# Define the input layer
inputs = tf.keras.Input(shape=(Xtrain.shape[1]))

# Define the hidden layers
hidden = tf.keras.layers.Dense(
    16, 
    activation='relu', 
    kernel_initializer=tf.keras.initializers.he_normal(seed=1),
    bias_initializer='zeros', 
    kernel_regularizer=tf.keras.regularizers.L2(0.01))(inputs)
hidden = tf.keras.layers.BatchNormalization()(hidden)
hidden = tf.keras.layers.Dense(
    32, activation='relu', 
    kernel_initializer=tf.keras.initializers.he_normal(seed=1),
    bias_initializer='zeros', 
    kernel_regularizer=tf.keras.regularizers.L2(0.01))(hidden)
hidden = tf.keras.layers.BatchNormalization()(hidden)
hidden = tf.keras.layers.Dense(
    16, 
    activation='relu', 
    kernel_initializer=tf.keras.initializers.he_normal(seed=1),
    bias_initializer='zeros', 
    kernel_regularizer=tf.keras.regularizers.L2(0.01))(hidden)
hidden = tf.keras.layers.BatchNormalization()(hidden)

# Define the output layer. Make sure the output dimension is correct.
outputs = tf.keras.layers.Dense(1, activation='relu')(hidden) # Ensures output is non-negative

# Create the model
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)


model.summary()


#Define the learning rate schedule
lr = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01,
    decay_steps=50,
    decay_rate=0.92,
    staircase=True
)

opt = tf.keras.optimizers.Adam(learning_rate=lr)


# Define a custom callback function
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_loss'] <0.21 and logs['loss']<0.21:
            self.model.stop_training = True

# Create an instance of the custom callback
custom_early_stopping = CustomEarlyStopping()


# Compile the model
model.compile(
    optimizer= opt, 
    loss=loss_wrapper(df['Uncertainty']),
#    metrics=['accuracy']
)
model_checkpoint = ModelCheckpoint('model.h5', 
                                   monitor='val_loss', 
                                   save_best_only=True)
# Train your model
history = model.fit(
    Xtrain,
    Ytrain,
    batch_size=64,
    epochs=1000,
    validation_data=(Xtest, Ytest),
    callbacks=[custom_early_stopping, model_checkpoint]
)

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 13)]              0         
                                                                 
 dense_88 (Dense)            (None, 16)                224       
                                                                 
 batch_normalization_66 (Bat  (None, 16)               64        
 chNormalization)                                                
                                                                 
 dense_89 (Dense)            (None, 32)                544       
                                                                 
 batch_normalization_67 (Bat  (None, 32)               128       
 chNormalization)                                                
                                                                 
 dense_90 (Dense)            (None, 16)                528

5/5 [==============================] - 0s 11ms/step - loss: 0.3977 - val_loss: 0.6193
Epoch 148/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.3474 - val_loss: 0.6071
Epoch 149/1000
5/5 [==============================] - 0s 25ms/step - loss: 0.3680 - val_loss: 0.5525
Epoch 150/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.3782 - val_loss: 0.5660
Epoch 151/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.3490 - val_loss: 0.6005
Epoch 152/1000
5/5 [==============================] - 0s 23ms/step - loss: 0.3350 - val_loss: 0.5464
Epoch 153/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.3520 - val_loss: 0.5480
Epoch 154/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.3471 - val_loss: 0.5760
Epoch 155/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.3648 - val_loss: 0.6126
Epoch 156/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.3254 - val_loss: 0.5552
Epoch

5/5 [==============================] - 0s 11ms/step - loss: 0.2738 - val_loss: 0.4278
Epoch 310/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.3054 - val_loss: 0.4107
Epoch 311/1000
5/5 [==============================] - 0s 23ms/step - loss: 0.2279 - val_loss: 0.3771
Epoch 312/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2493 - val_loss: 0.3893
Epoch 313/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2570 - val_loss: 0.3858
Epoch 314/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2561 - val_loss: 0.3876
Epoch 315/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2653 - val_loss: 0.4002
Epoch 316/1000
5/5 [==============================] - 0s 23ms/step - loss: 0.2690 - val_loss: 0.3726
Epoch 317/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2326 - val_loss: 0.4117
Epoch 318/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2612 - val_loss: 0.3965
Epoch

5/5 [==============================] - 0s 11ms/step - loss: 0.2204 - val_loss: 0.3694
Epoch 472/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2102 - val_loss: 0.3687
Epoch 473/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2238 - val_loss: 0.3700
Epoch 474/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2185 - val_loss: 0.3686
Epoch 475/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2280 - val_loss: 0.3649
Epoch 476/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2038 - val_loss: 0.3613
Epoch 477/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2619 - val_loss: 0.3579
Epoch 478/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2208 - val_loss: 0.3573
Epoch 479/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2812 - val_loss: 0.3531
Epoch 480/1000
5/5 [==============================] - 0s 22ms/step - loss: 0.2298 - val_loss: 0.3492
Epoch

5/5 [==============================] - 0s 11ms/step - loss: 0.2198 - val_loss: 0.3745
Epoch 634/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1815 - val_loss: 0.3741
Epoch 635/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1811 - val_loss: 0.3720
Epoch 636/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1736 - val_loss: 0.3695
Epoch 637/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1982 - val_loss: 0.3683
Epoch 638/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.2315 - val_loss: 0.3690
Epoch 639/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2139 - val_loss: 0.3689
Epoch 640/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1924 - val_loss: 0.3678
Epoch 641/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2162 - val_loss: 0.3676
Epoch 642/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1915 - val_loss: 0.3679
Epoch

5/5 [==============================] - 0s 11ms/step - loss: 0.2049 - val_loss: 0.3661
Epoch 796/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1886 - val_loss: 0.3664
Epoch 797/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1803 - val_loss: 0.3665
Epoch 798/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2199 - val_loss: 0.3660
Epoch 799/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2255 - val_loss: 0.3663
Epoch 800/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1980 - val_loss: 0.3660
Epoch 801/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2862 - val_loss: 0.3656
Epoch 802/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2358 - val_loss: 0.3653
Epoch 803/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2261 - val_loss: 0.3656
Epoch 804/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2090 - val_loss: 0.3652
Epoch

5/5 [==============================] - 0s 11ms/step - loss: 0.2178 - val_loss: 0.3649
Epoch 958/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2282 - val_loss: 0.3648
Epoch 959/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2603 - val_loss: 0.3645
Epoch 960/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1770 - val_loss: 0.3646
Epoch 961/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2343 - val_loss: 0.3645
Epoch 962/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1995 - val_loss: 0.3645
Epoch 963/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2180 - val_loss: 0.3650
Epoch 964/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2397 - val_loss: 0.3651
Epoch 965/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1966 - val_loss: 0.3650
Epoch 966/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.2352 - val_loss: 0.3648
Epoch

### Evaluating model performance on both the training and test data.

In [10]:
train_hwmquality = df.loc[X_train_indices, 'Uncertainty']
test_hwmquality = df.loc[X_test_indices, 'Uncertainty']
Ytrain_pred=model.predict(Xtrain)
Ytest_pred=model.predict(Xtest)

3/3 [==============================] - 0s 1000us/step


In [20]:
def custom_r2_score(y_true, y_pred, hwmquality):
    lower_bounds = y_true - hwmquality
    upper_bounds = y_true + hwmquality

    # Calculate the squared error
    mean= np.mean(y_true)
    squared_error = np.square(mean - y_true)

    # Mask values within the range
    masked_squared_error = np.minimum(np.square(y_pred - lower_bounds), np.square(y_pred - upper_bounds))

    # Calculate the custom R-squared
    total_error = np.sum(squared_error)
    total_masked_error = np.sum(masked_squared_error)


    if total_error == 0:
        custom_r2 = 1  # Handle potential division by zero
    else:
        custom_r2 = 1 - (total_masked_error / total_error)

    return custom_r2

train_custom_r2 = custom_r2_score(Ytrain, Ytrain_pred.flatten(), train_hwmquality)
test_custom_r2 = custom_r2_score(Ytest, Ytest_pred.flatten(), test_hwmquality)

print("Train Custom R-squared: {:.2f}".format(train_custom_r2))
print("Test Custom R-squared: {:.2f}".format(test_custom_r2))


Train Custom R-squared: 0.97
Test Custom R-squared: 0.83


In [12]:
def custom_mae(y_true, y_pred, hwmquality):
    lower_bounds = y_true - hwmquality
    upper_bounds = y_true + hwmquality
    masked_absolute_error = np.minimum(np.abs(y_pred - lower_bounds), np.abs(y_pred - upper_bounds))
    return np.mean(masked_absolute_error)

train_custom_mae = custom_mae(Ytrain, Ytrain_pred.flatten(), train_hwmquality)
test_custom_mae = custom_mae(Ytest, Ytest_pred.flatten(), test_hwmquality)

print("Train Custom MAE: {:.2f}".format(train_custom_mae))
print("Test Custom MAE: {:.2f}".format(test_custom_mae))

Train Custom MAE: 0.10
Test Custom MAE: 0.30


In [13]:
def custom_median_absolute_error(y_true, y_pred, hwmquality):
    lower_bounds = y_true - hwmquality
    upper_bounds = y_true + hwmquality
    masked_absolute_error = np.minimum(np.abs(y_pred - lower_bounds), np.abs(y_pred - upper_bounds))
    return np.median(masked_absolute_error)

train_custom_median = custom_median_absolute_error(Ytrain, Ytrain_pred.flatten(), train_hwmquality)
test_custom_median = custom_median_absolute_error(Ytest, Ytest_pred.flatten(), test_hwmquality)

print("Train Custom MDAE: {:.2f}".format(train_custom_median))
print("Test Custom MDAE: {:.2f}".format(test_custom_median))

Train Custom MDAE: 0.05
Test Custom MDAE: 0.12


In [14]:
def custom_nrmse(y_true, y_pred, hwmquality):
    lower_bounds = y_true - hwmquality
    upper_bounds = y_true + hwmquality
    masked_squared_error = np.minimum(np.square(y_pred - lower_bounds), np.square(y_pred - upper_bounds))
    mean_squared_error = np.mean(masked_squared_error)
    mean_true = np.mean(y_true)
    return np.sqrt(mean_squared_error) *100 / mean_true

train_custom_nrmse = custom_nrmse(Ytrain, Ytrain_pred.flatten(), train_hwmquality)
test_custom_nrmse = custom_nrmse(Ytest, Ytest_pred.flatten(), test_hwmquality)

print("Train Custom NRMSE: {:.2f}".format(train_custom_nrmse))
print("Test Custom NRMSE: {:.2f}".format(test_custom_nrmse))

Train Custom NRMSE: 19.56
Test Custom NRMSE: 59.18


In [15]:
from sklearn import metrics
from sklearn.metrics import r2_score

print("R-squared for training dataset: {:.2f}".format(r2_score(Ytrain, Ytrain_pred)))
print("R-squared for test dataset: {:.2f}".format(r2_score(Ytest, Ytest_pred)))
print("MAE for train: {:.2f}".format(metrics.mean_absolute_error(Ytrain, Ytrain_pred)))
print("MAE for test: {:.2f}".format(metrics.mean_absolute_error(Ytest, Ytest_pred)))

R-squared for training dataset: 0.97
R-squared for test dataset: 0.82
MAE for train: 0.11
MAE for test: 0.32


### Saving the model

In [16]:
# Specify the path and filename for the model
model_path = 'Scenario3.h5'

# Use the .save() method to save your model
model.save(model_path)

print(f"Model saved to {model_path}")


Model saved to Scenario3.h5


### Evaluating stream gauges, HWMs, and stream gagues + HWMs

In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error, mean_squared_error

def evaluate_model(predictions, actuals, model_name):
    # Flatten predictions if necessary
    predictions = predictions
    
    # Calculate metrics
    r_squared = np.round(r2_score(actuals, predictions), 2)
    mae = mean_absolute_error(actuals, predictions)
    mdae = median_absolute_error(actuals, predictions)
    nrmse = np.sqrt(mean_squared_error(actuals, predictions)) * 100 / np.mean(actuals)
    bias = np.mean(predictions - actuals)
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    
    # Prepare DataFrame
    errors_df = pd.DataFrame({
        "Metric": ["R-squared", "MAE", "Median Absolute Error", "NRMSE", "BIAS", "RMSE"],
        "Value": [r_squared, mae, mdae, nrmse, bias, rmse]
    })
    
    # Print the DataFrame to check
    print(f"Errors for {model_name}:")
    print(errors_df)

    # Save to CSV
    errors_df.to_csv(f"errors_{model_name}.csv", index=False)

    return errors_df


In [21]:
# Conversion factor from some unit to meters, assuming 'Flood' is initially in some other units
y_t = df['Flood'] * 0.3048 

# Ensure 'features' contains the correct column names you want to use for the model
x_t = df[features].copy()

# Scale the data
x_t_scaled = scaler.fit_transform(x_t)

# Assuming 'best_model' is already trained and ready to be used for predictions
y_t_pred = model.predict(x_t_scaled).flatten()


errors_df_t = evaluate_model(y_t_pred, y_t, "Overall_S3")


# First, add y_t and y_t_pred to your DataFrame if they aren't already included
df['y_t'] = y_t
df['y_t_pred'] = y_t_pred



# Correct filtering of the DataFrame based on 'Type'
df_strm = df[df['Type'] == 'Stream']

# Conversion factor from some unit to meters, assuming 'Flood' is initially in some other units
y_strm = df_strm['y_t']  
y_strm_pred = df_strm['y_t_pred'] 

errors_df_strm = evaluate_model(y_strm_pred, y_strm, "Stream_S3")


# Correct filtering of the DataFrame based on 'Type'
df_HWM = df[df['Type'] == 'HWMs']

# Conversion factor from some unit to meters, assuming 'Flood' is initially in some other units
y_HWM = df_HWM['y_t']  
y_HWM_pred = df_HWM['y_t_pred'] 

errors_df_HWM = evaluate_model(y_HWM_pred, y_HWM, "HWMs_S3")

12/12 [==============================] - 0s 2ms/step
Errors for Overall_S3:
                  Metric      Value
0              R-squared   0.930000
1                    MAE   0.151360
2  Median Absolute Error   0.060111
3                  NRMSE  32.025584
4                   BIAS  -0.037145
5                   RMSE   0.309886
Errors for Stream_S3:
                  Metric      Value
0              R-squared   0.910000
1                    MAE   0.227894
2  Median Absolute Error   0.059327
3                  NRMSE  24.120999
4                   BIAS  -0.067035
5                   RMSE   0.473550
Errors for HWMs_S3:
                  Metric      Value
0              R-squared   0.840000
1                    MAE   0.116545
2  Median Absolute Error   0.060960
3                  NRMSE  37.723546
4                   BIAS  -0.023548
5                   RMSE   0.194170
